<a href="https://colab.research.google.com/github/hdtruong802/Pytorch-Assignments/blob/main/%5BNLP%5DPart2_Build_GPTModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dự án thực hành
```
ProtonX - Pytorch Class
```

### Hướng dẫn làm bài
- Trong bài tập này bạn sẽ sử dụng Python 3.
- Sau khi bạn viết Code của mình xong, hãy chạy dòng Code đó để xem kết quả bên dưới.

### [Quan trọng] Chú ý
- **Không sử dụng hàm `input()` tại bất kỳ dòng lệnh nào**
- **Không thay đổi dòng code return của hàm**

Các bạn sẽ thực hiện `code` trong các phần hiển thị `#TODO: Lập trình tại đây` và thay thế các vị trí `None`. Có những câu hỏi chỉ cần trả về đáp án.

Sau khi viết xong Code của bạn, bạn hãy ấn "SHIFT"+"ENTER" để thực hiện chạy lệnh của Cell đó.

---
Điểm số:
* 10 điểm / Câu

Tiêu chí chấm điểm:
* Các bài tập sẽ được chấm dựa trên các Test-case.
* Các bạn không khởi tạo lại giá trị đầu vào bên trong hàm. Có thể khởi tạo các giá trị này ngoài hàm nhằm mục đích kiểm thử.

## Phần 1: Tách token

**Cài đặt bộ tokenizer bên thứ 3**

Xem thêm về các bộ tách từ xây dựng bằng [Spacy](https://protonx.io/courses/6523788337e7f90012890145/topics/6571b351e5bfd8001a8f0d50).

In [1]:
from torch.nn.utils.rnn import pad_sequence

In [2]:
!pip install pyvi
from spacy.lang.vi import Vietnamese

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.0 MB/s eta 0:00:00


Thử tokenizer này

In [3]:
nlp = Vietnamese()
tokens = nlp('Lớp học Pytorch là lớp học rất hay ho')

In [4]:
tokens

Lớp học Pytorch là lớp học rất hay ho

Lặp qua các đơn vị lưu trữ trong Vocab dưới dạng từ vựng (Lexeme). Một từ vựng có các thuộc tính sau:

In [5]:
for word in tokens:
    lexeme = tokens.vocab[word.text]
    print(lexeme.text, lexeme.orth, lexeme.shape_, lexeme.prefix_, lexeme.suffix_,
            lexeme.is_alpha, lexeme.is_digit, lexeme.is_title, lexeme.lang_)

Lớp học 9560392879819997953 Xxx xxx L học False False False vi
Pytorch 16295890598922194816 Xxxxx P rch True False True vi
là 12844001651356581174 xx l là True False False vi
lớp học 16939850666758009144 xxx xxx l học False False False vi
rất 1932685823275808203 xxx r rất True False False vi
hay ho 3393752619603724165 xxx xx h  ho False False False vi


- Văn bản: Văn bản gốc của từ vựng.

- Orth: Giá trị băm của từ vựng.

- Shape: Hình dạng từ ngữ trừu tượng của từ vựng.

- Prefix: Theo mặc định, là chữ cái đầu tiên của chuỗi từ.

- Suffix: Theo mặc định, là ba chữ cái cuối cùng của chuỗi từ.

- is alpha: Liệu từ vựng có bao gồm các ký tự chữ cái không?

- is digit: Liệu từ vựng có bao gồm các chữ số không?

Phân tích: Có rất nhiều câu ngắn trong dữ liệu vì thế bạn hãy lọc những câu có chiều dài ngắn hơn 10 từ và tiến hành tách token

In [6]:
!pwd

/content


In [7]:
%%writefile wiki.txt
Những gì bạn sẽ học được từ lớp học này

Hiểu cách các mô hình học sâu hoạt động

Triển khai các model tiên tiến nhất với Pytorch bao gồm thị giác máy tính và xử lý ngôn ngữ tự nhiên

Cách làm việc với nhiều loại dữ liệu khác nhau để xây dựng model

Writing wiki.txt


In [8]:
from collections import Counter
import io

def count_tokens_in_file(filepath, min_length):
    """
    Hàm này đọc một tệp tin văn bản và đếm số lượng xuất hiện của từng token.
    Chỉ xử lý các câu có chiều dài lớn hơn hoặc bằng min_length.

    Args:
    filepath: Đường dẫn đến tệp tin văn bản.
    min_length: Độ dài tối thiểu của câu để được xử lý.

    Returns:
    Counter object với số lượng xuất hiện của mỗi token.
    """

    # Bộ đếm token và số lượng xuất hiện
    counter = Counter()

    # Mở File
    with io.open(filepath, encoding="utf8") as f:
        for string_ in f:
            # Chỉ làm việc với những câu có chiều dài lớn hơn hoặc bằng min_length
            # Tiến hành tách token
            doc = nlp(string_)
            # Update counter với bộ đếm
            counter.update([token.text for token in doc])
    return counter

# Sử dụng hàm
filepath = '/content/wiki.txt'
min_length = 10
counter = count_tokens_in_file(filepath, min_length)

Từ bộ đếm này, xây dựng từ điển bằng Pytorch



In [9]:
# Token không có
unk_token = '<unk>'
# Token để padding
pad_token = '<pad>'
# Token bắt đầu câu
bos_token = '<bos>'
# Token kết thúc câu
eos_token = '<eos>'

from torchtext.vocab import vocab
# TODO 5: Xây dựng Vocab bằng Pytorch
torchVocab = vocab(counter, specials=[unk_token, pad_token , bos_token, eos_token])

Test Code

In [10]:
def test_initialization():
    try:
        assert unk_token in torchVocab.get_stoi()
        assert pad_token in torchVocab.get_stoi()
        assert bos_token in torchVocab.get_stoi()
        assert eos_token in torchVocab.get_stoi()
        print("Test Initialization: Passed")
    except AssertionError:
        print("Test Initialization: Failed")

def test_special_tokens():
    try:
        for token in [unk_token, pad_token, bos_token, eos_token]:
            assert token in torchVocab.get_stoi()
        print("Test Special Tokens: Passed")
    except AssertionError:
        print("Test Special Tokens: Failed")

def test_token_to_index():
    try:
        known_token = list(counter.keys())[0]
        assert torchVocab[known_token] == torchVocab.get_stoi()[known_token]
        assert torchVocab['<unk>'] == torchVocab.get_stoi()[str(unk_token)]
        print("Test Token to Index Conversion: Passed")
    except AssertionError:
        print("Test Token to Index Conversion: Failed")

def test_index_to_token():
    try:
        known_index = torchVocab.get_stoi()[list(counter.keys())[0]]
        assert torchVocab.get_itos()[known_index] == list(counter.keys())[0]
        print("Test Index to Token Conversion: Passed")
    except AssertionError:
        print("Test Index to Token Conversion: Failed")

def test_vocabulary_size():
    try:
        expected_size = len(counter) + 4  # 4 for the special tokens
        assert len(torchVocab) == expected_size
        print("Test Vocabulary Size: Passed")
    except AssertionError:
        print("Test Vocabulary Size: Failed")

def test_unknown_token_handling():
    try:
        assert torchVocab['<unk>'] == torchVocab.get_stoi()[unk_token]
        print("Test Handling of Unknown Tokens: Passed")
    except AssertionError:
        print("Test Handling of Unknown Tokens: Failed")


test_initialization()
test_special_tokens()
test_token_to_index()
test_index_to_token()
test_vocabulary_size()
test_unknown_token_handling()


Test Initialization: Passed
Test Special Tokens: Passed
Test Token to Index Conversion: Passed
Test Index to Token Conversion: Passed
Test Vocabulary Size: Passed
Test Handling of Unknown Tokens: Passed


**Kết quả mong đợi:**

```
Test Initialization: Passed
Test Special Tokens: Passed
Test Token to Index Conversion: Passed
Test Index to Token Conversion: Passed
Test Vocabulary Size: Passed
Test Handling of Unknown Tokens: Passed
```



In [11]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 4
PAD_IDX = torchVocab['<pad>']
BOS_IDX = torchVocab['<bos>']
EOS_IDX = torchVocab['<eos>']

### Lập trình hàm cắt một chuỗi thành nhiều chuỗi nhỏ

Hàm `trim_sequence_to_parts` có chức năng chia một chuỗi (hoặc một dãy các phần tử) thành nhiều phần nhỏ với độ dài xác định. Dưới đây là mô tả chi tiết về các đối số đầu vào và giá trị trả về của hàm này:

**Đối số:**
1. `sequence`: Đây là chuỗi hoặc dãy các phần tử cần được chia nhỏ. Đối số này có thể là một chuỗi các ký tự, một danh sách, hoặc bất kỳ đối tượng nào khác có thể được lặp qua (iterable).
2. `part_length`: Đây là độ dài của từng phần sau khi chia. Đối số này phải là một số nguyên, chỉ ra số lượng phần tử tối đa trong mỗi phần nhỏ sau khi chia.

**Chức năng của Hàm:**
- Hàm `trim_sequence_to_parts` lặp qua `sequence` và chia nó thành các phần nhỏ, mỗi phần có độ dài không vượt quá `part_length`.
- Việc chia này được thực hiện bằng cách sử dụng list comprehension, một cấu trúc ngắn gọn trong Python để tạo ra các danh sách mới.

**Giá trị trả về:**
- Hàm trả về một danh sách mới, trong đó mỗi phần tử là một phần của `sequence` với độ dài không vượt quá `part_length`.
- Nếu `sequence` không thể chia đều cho `part_length`, phần cuối cùng của danh sách trả về có thể sẽ ngắn hơn.

**Ví dụ:**
Nếu bạn gọi `trim_sequence_to_parts([1, 2, 3, 4, 5, 6, 7], 3)`, hàm sẽ trả về `[[1, 2, 3], [4, 5, 6], [7]]`. Trong ví dụ này, chuỗi `[1, 2, 3, 4, 5, 6, 7]` được chia thành các phần, mỗi phần có 3 phần tử, trừ phần cuối cùng chỉ có 1 phần tử.

In [12]:
max_length = 30
def trim_sequence_to_parts(sequence, part_length):
    # TODO 5: Lập trình tại đây
    return [sequence[i:i + part_length] for i in range(0, len(sequence), part_length)]

### Lập trình hàm xử lý dữ liệu

Hàm `data_process` là một hàm dùng để xử lý dữ liệu văn bản từ một tệp tin và chuyển đổi chúng thành dạng tensor sử dụng một từ điển (vocab). Dưới đây là mô tả chi tiết về các đối số và giá trị trả về của hàm này:

**Đối số:**
1. `filepath`: Đường dẫn đến tệp tin chứa dữ liệu văn bản cần xử lý. Đây là một chuỗi ký tự (string) chỉ ra vị trí của tệp tin trong hệ thống tập tin.
2. `vocab`: Một từ điển hoặc cấu trúc dữ liệu tương tự, dùng để chuyển đổi từ các token văn bản sang số nguyên (index). Mỗi token trong văn bản sẽ được tìm kiếm trong `vocab` để lấy chỉ số tương ứng.
3. `min_length`: Độ dài tối thiểu của một chuỗi ký tự (câu) để nó được xem xét và xử lý. Nếu một chuỗi ngắn hơn `min_length`, nó sẽ bị bỏ qua.
4. `max_length`: Độ dài tối đa cho phép của một chuỗi tensor sau khi được chia nhỏ bằng hàm `trim_sequence_to_parts`.

**Chức năng của Hàm:**
- Hàm mở tệp tin văn bản tại `filepath` sử dụng mã hóa UTF-8.
- Đọc từng dòng trong tệp tin. Mỗi dòng được coi như một chuỗi ký tự (`string_`).
- Nếu độ dài của `string_` (tính theo số lượng từ) lớn hơn hoặc bằng `min_length`, chuỗi sẽ được xử lý tiếp:
    - Sử dụng một hàm nlp như trên để xử lý chuỗi ký tự này, tạo ra một đối tượng (có thể là một danh sách các token).
    - Mỗi token trong đối tượng này được chuyển đổi thành số nguyên (index) sử dụng `vocab`. Kết quả là một tensor PyTorch (`vi_tensor_`).
    - Tensor này sau đó được chia thành nhiều phần nhỏ hơn với độ dài tối đa là `max_length` sử dụng hàm `trim_sequence_to_parts`.
    - Mỗi phần nhỏ này được thêm vào danh sách `data`.

**Giá trị trả về:**
- Hàm trả về `data`, là một danh sách các phần tensor. Mỗi phần tensor đại diện cho một phần của dữ liệu văn bản, đã được chuyển đổi thành dạng số nguyên và chia theo độ dài tối đa xác định.



In [13]:
import traceback

def data_process(filepath, vocab, min_length, max_length):
    data = []
    # TODO 6: Lập trình tại đây
    with io.open(filepath, encoding="utf8") as f:
        for string_ in f:
            if len(string_.split()) >= min_length:
                doc = nlp(string_)
                vi_tensor_ = torch.tensor([vocab[str(token)] for token in doc], dtype=torch.long)
                parts = trim_sequence_to_parts(vi_tensor_, max_length)
                for part in parts:
                    data.append(part)
    return data

In [14]:
train_data = data_process(filepath, torchVocab, 10, 30)

Test Code

In [15]:
%%writefile test.txt
Những gì bạn sẽ học được từ lớp học này

Hiểu cách các mô hình học sâu hoạt động

Triển khai các model tiên tiến nhất với Pytorch bao gồm thị giác máy tính và xử lý ngôn ngữ tự nhiên

Cách làm việc với nhiều loại dữ liệu khác nhau để xây dựng model

Writing test.txt


In [16]:
file_test_path = 'test.txt'
test_counter = count_tokens_in_file(file_test_path, min_length)
testVocab = vocab(test_counter, specials=[unk_token, pad_token , bos_token, eos_token])
test_data = data_process(file_test_path, testVocab, 10, 30)

In [17]:
test_counter

Counter({'Những': 1,
         'gì': 1,
         'bạn': 1,
         'sẽ': 1,
         'học': 1,
         'được': 1,
         'từ': 1,
         'lớp học': 1,
         'này': 1,
         '\n': 7,
         'Hiểu': 1,
         'cách': 1,
         'các': 2,
         'mô hình học': 1,
         'sâu': 1,
         'hoạt động': 1,
         'Triển khai': 1,
         'model': 2,
         'tiên tiến': 1,
         'nhất': 1,
         'với': 2,
         'Pytorch': 1,
         'bao gồm': 1,
         'thị giác': 1,
         'máy tính': 1,
         'và': 1,
         'xử lý': 1,
         'ngôn ngữ': 1,
         'tự nhiên': 1,
         'Cách': 1,
         'làm việc': 1,
         'nhiều': 1,
         'loại': 1,
         'dữ liệu': 1,
         'khác': 1,
         'nhau': 1,
         'để': 1,
         'xây dựng': 1})

In [18]:
test_data

[tensor([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),
 tensor([20, 16, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 13]),
 tensor([33, 34, 24, 35, 36, 37, 38, 39, 40, 41, 21, 13])]

Kết quả mong đợi

```python

[tensor([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),
 tensor([20, 16, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 13]),
 tensor([33, 34, 24, 35, 36, 37, 38, 39, 40, 41, 21, 13])]
```

In [19]:
BATCH_SIZE = 4
PAD_IDX = torchVocab['<pad>']
BOS_IDX = torchVocab['<bos>']
EOS_IDX = torchVocab['<eos>']

In [20]:
EOS_IDX

3

### Lập trình hàm chia batch

Hàm `generate_batch` là một hàm được thiết kế để xử lý các batch dữ liệu (batch) và chuẩn bị chúng để có thể được sử dụng trong các mô hình học máy, đặc biệt là trong xử lý ngôn ngữ tự nhiên. Dưới đây là mô tả chi tiết về các đối số và giá trị trả về của hàm này:

**Đối số:**
1. `data_batch`: Một batch dữ liệu, thường là một danh sách các tensor. Mỗi tensor trong `data_batch` đại diện cho một mẫu dữ liệu (ví dụ, một câu hoặc đoạn văn).

**Chức năng của Hàm:**
- Hàm lặp qua mỗi mẫu dữ liệu (`vi_item`) trong `data_batch`.
- Đối với mỗi `vi_item`:
  - Tạo một tensor `pads` chứa các giá trị padding (PAD_IDX) với kích thước sao cho tổng chiều dài của `vi_item` sau khi thêm các giá trị padding sẽ bằng `max_length`.
  - Sử dụng hàm `torch.cat` để ghép `vi_item` với các giá trị BOS_IDX (đánh dấu bắt đầu của chuỗi), EOS_IDX (đánh dấu kết thúc của chuỗi), và tensor `pads` chứa các giá trị padding.
  - Thêm `vi_item` đã được xử lý vào danh sách `vi_batch`.
- Cuối cùng, sử dụng hàm `pad_sequence` để chuẩn hóa kích thước của tất cả các mẫu trong `vi_batch`, đảm bảo rằng tất cả chúng có chiều dài như nhau bằng cách thêm các giá trị padding (PAD_IDX) ở cuối nếu cần.

**Giá trị trả về:**
- Hàm trả về `vi_batch`, là một tensor đã được chuẩn hóa về kích thước, với mỗi mẫu dữ liệu trong batch đều có cùng chiều dài `max_length`.


In [21]:
def generate_batch(data_batch):
  vi_batch = []

  # TODO 7: Lập trình tại đây
  for vi_item in data_batch:
      pads = torch.full(size=(max_length - vi_item.shape[0],), fill_value=PAD_IDX, dtype=torch.long)
      vi_item = torch.cat([torch.tensor([BOS_IDX]), vi_item, torch.tensor([EOS_IDX]), pads], dim=0)
      vi_batch.append(vi_item)
  vi_batch = pad_sequence(vi_batch, padding_value=PAD_IDX, batch_first=True)
  return vi_batch


Test Code

In [22]:
def test_generate_batch():
    try:
        # Test Case 1: Single item in data_batch
        data_batch = [torch.tensor([3, 4, 5])]
        batch = generate_batch(data_batch)
        assert batch.shape == (1, 32), "Failed: Shape mismatch in single item test"

        # Test Case 2: Multiple items in data_batch of different lengths
        data_batch = [torch.tensor([3, 4, 5]), torch.tensor([6, 7])]
        batch = generate_batch(data_batch)
        assert batch.shape == (2, 32), "Failed: Shape mismatch in multiple items test"

        # Test Case 3: Check if BOS_IDX and EOS_IDX are added correctly
        data_batch = [torch.tensor([3])]
        batch = generate_batch(data_batch)
        assert batch[0][0] == BOS_IDX and batch[0][2] == EOS_IDX, "Failed: BOS_IDX or EOS_IDX missing"

        print("All tests passed.")
    except AssertionError as e:
        print(e)

# Run the tests
test_generate_batch()


All tests passed.


**Kết quả mong đợi:**

All tests passed.


In [23]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
test_iter = DataLoader(test_data, batch_size=4,
                        shuffle=True, collate_fn=generate_batch)
for item in test_iter:
    print('Chiều: ', item.shape)
    print('Giá trị')
    print(item)

Chiều:  torch.Size([3, 32])
Giá trị
tensor([[ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  3,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2, 33, 34, 24, 35, 36, 37, 38, 39, 40, 41, 21, 13,  3,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2, 20, 16, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 13,  3,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1]])


**Kết quả mong đợi:**



```
Chiều:  torch.Size([3, 32])
Giá trị
tensor([[ 2, 33, 34, 24, 35, 36, 37, 38, 39, 40, 41, 21, 13,  3,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  3,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [ 2, 20, 16, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 13,  3,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1]])
```



Thực hiện build trên bộ wiki lớn

In [24]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
train_iter = DataLoader(train_data, batch_size=8,
                        shuffle=True, collate_fn=generate_batch)

# for item in train_iter:
#     print('Chiều: ', item.shape)
#     print('Giá trị')
#     print(item)

Save vocab

In [25]:
torch.save(torchVocab, 'wikiVocab.pth')

Load vocab

In [26]:
torchVocab = torch.load('wikiVocab.pth')

In [27]:
# torchVocab["tiên tiến"]

## Phần 2: Xây dựng mô hình GPT Decoder
---
![](https://storage.googleapis.com/protonx-cdn/Screen%20Shot%202024-01-29%20at%2015.26.02.png)

Để hiểu được mô hình, xem chi tiết video buổi hướng dẫn phần 2 ở trong lớp học


**Kiến trúc lớp Decoder**
- Lớp MultiheadAttention
- Sau đó đi qua các lớp Layer Norm và Dropout

In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class GPTDecoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1):
        super(GPTDecoderLayer, self).__init__()
        # TODO 1: Tạo cơ chế Multihead Attention với chiều embedding là d_model, số head là nhead và tỷ lệ dropout đã chỉ định
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        # TODO 2: Tạo phép biến đổi tuyến tính đầu tiên
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        # TODO 3: Thêm dropout
        self.dropout = nn.Dropout(dropout)
        # TODO 4: Tạo phép biến đổi tuyến tính thứ hai
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        # TODO 5: Tạo phép LayerNorm 1
        self.norm1 = nn.LayerNorm(d_model)

        # TODO 6: Tạo phép LayerNorm 2
        self.norm2 = nn.LayerNorm(d_model)

        # TODO 7: Tạo phép Dropout 1
        self.dropout1 = nn.Dropout(dropout)

        # TODO 8: Tạo phép Dropout 2
        self.dropout2 = nn.Dropout(dropout)

        # TODO 9: Tạo phép activation ReLU
        self.activation = F.relu

    def forward(self, src, src_mask=None, src_key_padding_mask=None):

        # TODO 10: Áp dụng cơ chế Multihead Attention
        src2, _ = self.self_attn(src, src, src, attn_mask=src_mask, key_padding_mask=src_key_padding_mask)

        # TODO 11: Áp dụng Dropout 1 và thêm kết quả vào đầu vào
        src = src + self.dropout1(src2)

        # TODO 12: Áp dụng LayerNorm 1
        src = self.norm1(src)

        # TODO 13: Áp dụng phép biến đổi tuyến tính - chú ý thêm activation
        src2 = self.linear2(self.dropout(self.activation(self.linear1(src))))

        # TODO 14: Áp dụng Dropout 2 và thêm kết quả vào đầu vào
        src = src + self.dropout2(src2)

        # TODO 15: Áp dụng LayerNorm 2
        src = self.norm2(src)
        return src


Test Code

In [32]:
def test_initialization():
    d_model = 512
    nhead = 8
    dim_feedforward = 2048
    dropout = 0.1
    decoder_layer = GPTDecoderLayer(d_model, nhead, dim_feedforward, dropout)
    assert isinstance(decoder_layer.self_attn, nn.MultiheadAttention), "self_attn is not an instance of MultiheadAttention"
    print('Passed')

def test_multihead_attention():
    d_model = 512
    nhead = 8
    sample_input = torch.rand(10, 32, d_model)
    multihead_attn = nn.MultiheadAttention(d_model, nhead)

    attn_output, _ = multihead_attn(sample_input, sample_input, sample_input)
    assert attn_output.shape == sample_input.shape, "Output shape of multihead attention is incorrect"
    print('Passed')



def test_forward():
    d_model = 512
    nhead = 8
    dim_feedforward = 2048
    dropout = 0.1
    decoder_layer = GPTDecoderLayer(d_model, nhead, dim_feedforward, dropout)
    sample_input = torch.rand(10, 32, d_model)

    output = decoder_layer(sample_input)
    assert output.shape == sample_input.shape, "Output shape in forward pass is incorrect"
    print('Passed')

# Run tests
test_initialization()
test_multihead_attention()
test_forward()


Passed
Passed
Passed


Kết quả mong đợi:

```python
Passed
Passed
Passed
```

**Kiến trúc mô hình GPT**
- Embedding của các token trong câu
- Embedding để học vị trí của các từ trong câu - Gọi là Positional Embedding
- Chuỗi các layer Decoder
 - Multihead Attention
 - Hàm chuẩn hóa Norm và Add
- Lớp Linear cuối cùng

In [33]:
class GPTModel(nn.Module):
    def __init__(self, num_layers, d_model, nhead, vocab_size, max_seq_length, dim_feedforward=2048, dropout=0.1):
        super(GPTModel, self).__init__()

        # TODO 16: Tạo embedding cho các token đầu vào.
        self.embedding = nn.Embedding(vocab_size, d_model)

        # TODO 17: Tạo embedding cho mã hóa vị trí
        self.pos_embedding = nn.Embedding(max_seq_length, d_model)

        # TODO 18: Khởi tạo một danh sách các đối tượng GPTDecoderLayer, tạo nên cốt lõi của mô hình GPT. Dùng hàm ModuleList
        self.layers = nn.ModuleList([GPTDecoderLayer(d_model, nhead, dim_feedforward, dropout) for _ in range(num_layers)])

        # TODO 19: Lớp tuyến tính cuối cùng để tìm ra từ tiếp theo
        self.linear = nn.Linear(d_model, vocab_size)

        self.d_model = d_model

    def forward(self, x):
        seq_length = x.size(1)

        # TODO 20: Tạo một mảng các vị trí từ 0 đến seq_length - 1
        positions = torch.arange(seq_length, device=x.device, dtype=torch.long)

        # TODO 21: Áp dụng embedding vị trí
        positions = self.pos_embedding(positions)

        # TODO 22: Nhân embedding của chuỗi token với căn bậc 2 của d_model
        x = self.embedding(x) * math.sqrt(self.d_model)

        # TODO 23: Cộng embedding của chuỗi token với embedding của vị trí
        x += positions

        # TODO 24: Đưa chuỗi kết quả qua các lớp Decoder
        for layer in self.layers:
            x = layer(x)

        # TODO 25: Đưa qua lớp Linear cuối cùng
        x = self.linear(x)
        return x

In [34]:
def test_initialization_with_valid_parameters():
    try:
        model = GPTModel(num_layers=2, d_model=512, nhead=8, vocab_size=10000, max_seq_length=50)
        print("Test passed: Initialization with valid parameters.")
    except Exception as e:
        print(f"Test failed: {e}")

def test_forward_pass_with_valid_input():
    try:
        model = GPTModel(num_layers=2, d_model=512, nhead=8, vocab_size=10000, max_seq_length=50)
        input_tensor = torch.randint(0, 10000, (1, 50))
        output = model(input_tensor)
        print("Test passed: Forward pass with valid input.")
    except Exception as e:
        print(f"Test failed: {e}")

def test_forward_pass_with_invalid_input_size():
    try:
        model = GPTModel(num_layers=2, d_model=512, nhead=8, vocab_size=10000, max_seq_length=50)
        input_tensor = torch.randint(0, 10000, (1, 51))  # Invalid size
        output = model(input_tensor)
        print("Test failed: Invalid input size should have caused an error.")
    except Exception as e:
        print("Test passed: Caught an error with invalid input size.")


def test_output_shape():
    try:
        model = GPTModel(num_layers=2, d_model=512, nhead=8, vocab_size=10000, max_seq_length=50)
        input_tensor = torch.randint(0, 10000, (1, 50))
        output = model(input_tensor)
        assert output.shape == (1, 50, 10000), "Output shape is incorrect."
        print("Test passed: Output shape is correct.")
    except Exception as e:
        print(f"Test failed: {e}")

def test_layer_counts():
    try:
        num_layers = 2
        model = GPTModel(num_layers=num_layers, d_model=512, nhead=8, vocab_size=10000, max_seq_length=50)
        assert len(model.layers) == num_layers, "Incorrect number of layers."
        print("Test passed: Layer count is correct.")
    except Exception as e:
        print(f"Test failed: {e}")


test_initialization_with_valid_parameters()
test_forward_pass_with_valid_input()
test_forward_pass_with_invalid_input_size()
test_output_shape()
test_layer_counts()

Test passed: Initialization with valid parameters.
Test passed: Forward pass with valid input.
Test passed: Caught an error with invalid input size.
Test passed: Output shape is correct.
Test passed: Layer count is correct.


Kết quả mong đợi

```python
Test passed: Initialization with valid parameters.
Test passed: Forward pass with valid input.
Test passed: Caught an error with invalid input size.
Test passed: Output shape is correct.
Test passed: Layer count is correct.
```

Khởi tạo mô hình

In [35]:
num_layers = 6
d_model = 512
nhead = 8
vocab_size = len(torchVocab)
max_seq_length = 32

# Tạo mô hình
model = GPTModel(num_layers, d_model, nhead, vocab_size, max_seq_length)

# Tạo mẫu đơn giản với chiều (batch_size, max_seq_length): (1, 32)
example_input = torch.randint(0, vocab_size, (1, 32))
output = model(example_input)

print(output.shape)  # Đầu ra [1, 32, vocab_size]

torch.Size([1, 32, 42])


In [36]:
# Hàm đánh giá
def evaluate(model, data_loader, criterion):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in data_loader:
            batch = batch.to(device)
            inputs, labels = batch[:, :-1], batch[:, 1:]
            outputs_reshaped = outputs.reshape(-1, outputs.size(-1))
            labels_reshaped = labels.reshape(-1)
            loss = criterion(outputs_reshaped, labels_reshaped)
            total_loss += loss.item()

    return total_loss / len(data_loader)

In [37]:
import torch
import torch.nn as nn
import torch.optim as optim


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Di chuyển model đến GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Tiến hành training
num_epochs = 20  # Số epochs
model.train()  # Chuyển chế độ thành train

for epoch in range(num_epochs):
    total_loss = 0

    for batch in train_iter:
        # Chuyển data sang device
        batch = batch.to(device)

        # Sắp xếp data để training
        inputs, labels = batch[:, :-1], batch[:, 1:]

        # Chiều xuôi
        outputs = model(inputs)

        outputs_reshaped = outputs.reshape(-1, outputs.size(-1))
        labels_reshaped = labels.reshape(-1)
        loss = criterion(outputs_reshaped, labels_reshaped)

        # Chiều ngược và cập nhật tham số
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    avg_loss = total_loss / len(train_iter)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

print("Training completed")


Epoch [1/20], Loss: 4.0686
Epoch [2/20], Loss: 3.7656
Epoch [3/20], Loss: 5.2404
Epoch [4/20], Loss: 1.7367
Epoch [5/20], Loss: 1.2405
Epoch [6/20], Loss: 1.0030
Epoch [7/20], Loss: 1.0999
Epoch [8/20], Loss: 1.0888
Epoch [9/20], Loss: 0.8094
Epoch [10/20], Loss: 0.5838
Epoch [11/20], Loss: 0.3621
Epoch [12/20], Loss: 0.2917
Epoch [13/20], Loss: 0.1923
Epoch [14/20], Loss: 0.1530
Epoch [15/20], Loss: 0.1360
Epoch [16/20], Loss: 0.1147
Epoch [17/20], Loss: 0.1003
Epoch [18/20], Loss: 0.0695
Epoch [19/20], Loss: 0.0784
Epoch [20/20], Loss: 0.0641
Training completed


**Kết quả mong đợi (Tương đối)**

```python
Epoch [1/20], Loss: 3.7509
Epoch [2/20], Loss: 3.8950
Epoch [3/20], Loss: 5.0087
Epoch [4/20], Loss: 1.6143
Epoch [5/20], Loss: 0.9752
Epoch [6/20], Loss: 0.7447
Epoch [7/20], Loss: 0.6585
Epoch [8/20], Loss: 0.5026
Epoch [9/20], Loss: 0.5309
Epoch [10/20], Loss: 0.4016
Epoch [11/20], Loss: 0.3491
Epoch [12/20], Loss: 0.2637
Epoch [13/20], Loss: 0.2202
Epoch [14/20], Loss: 0.1943
Epoch [15/20], Loss: 0.1399
Epoch [16/20], Loss: 0.1155
Epoch [17/20], Loss: 0.0927
Epoch [18/20], Loss: 0.0733
Epoch [19/20], Loss: 0.0730
Epoch [20/20], Loss: 0.0586
Training completed
```

Thử decode

In [42]:
doc = nlp('Những')
vi_tensor_ = torch.tensor([torchVocab[str(token)] for token in doc], dtype=torch.long)
vi_item = torch.cat([torch.tensor([BOS_IDX]), vi_tensor_], dim=0)
input_ids = torch.tensor(vi_item, dtype=torch.long).unsqueeze(0)
input_ids = input_ids.to(device)

for _ in range(max_length):
    outputs = model(input_ids)
    predictions = outputs[:, -1, :].argmax(dim=-1)
    pred_item = predictions.unsqueeze(-1)
    input_ids = torch.cat([input_ids, pred_item], dim=-1)
    if pred_item == EOS_IDX:
        break
    print(torchVocab.lookup_token(pred_item))


<ipython-input-42-8e44b1ae2302>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(vi_item, dtype=torch.long).unsqueeze(0)


gì
bạn
sẽ
học
được
từ
lớp học
này


